`05_websearch`

# Tavily Web Search

- LLM에게 웹검색을 쥐어주자

In [1]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [6]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [7]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': '60822112-6e1a-428b-985d-b8fb49663ee6',
 'response_time': 1.31,
 'results': [{'content': '강남신세계 혼밥맛집 (106곳) · 1. 소녀방앗간 서울고속터미널점 · 2. 타코벨 '
                         '서울고속버스터미널점 · 3. 소이연남 파미에스테이션점 · 4. 갓덴스시 센트럴시티점 · 5. '
                         '101번지',
              'raw_content': None,
              'score': 0.7304634,
              'title': "'강남신세계 혼밥' 맛집 빅데이터 추천순위 Top100",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%20%ED%98%BC%EB%B0%A5'},
             {'content': '1. 더 마고 그릴 신세계강남 · 2. 모던눌랑 강남고속터미널 · 3. 에토레 파미에스테이션 '
                         '· 4. 자주테이블 신세계강남 · 5. 쓰리버즈 센트럴시티점 · 6. 더 키친 일뽀르노 신세계',
              'raw_content': None,
              'score': 0.6907474,
              'title': "'강남신세계백화점 레스토랑' 맛집 빅데이터 추천순위 Top10",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8

In [8]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

"강남 신세계백화점에서 점심을 맛있게 즐기고 싶다면, '타치바나'를 추천합니다. 신세계백화점 지하 1층에 위치해 있으며, 다양한 맛집과 다이닝 옵션도 있습니다. 특히 미쉐린 빕 구르망 셰프의 코스 메뉴도 마련되어 있어 점심 식사에 적합합니다."

In [9]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 내에서는 다양한 맛집이 있지만, 점심 식사에 추천할 만한 곳으로는 셰프가 엄선한 메뉴를 제공하는 레스토랑이나, 캐주얼한 분위기의 카페들이 있습니다. 특히, 신세계백화점 지하에 위치한 푸드코트에서는 다양한 메뉴를 저렴하게 즐기실 수 있어 편리합니다.'

In [16]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose=True)

In [17]:
agent_executor.invoke({'input': '나는 춘식이야'})



> Entering new AgentExecutor chain...
반갑습니다, 춘식이님! 오늘 도와드릴 일이 있나요? 어떤 것이든 말씀해 주세요.

> Finished chain.


{'input': '나는 춘식이야',
 'chat_history': [HumanMessage(content='나는 춘식이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반갑습니다, 춘식이님! 오늘 도와드릴 일이 있나요? 어떤 것이든 말씀해 주세요.', additional_kwargs={}, response_metadata={})],
 'output': '반갑습니다, 춘식이님! 오늘 도와드릴 일이 있나요? 어떤 것이든 말씀해 주세요.'}